In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, load_img
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pylab as plt
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score,recall_score,accuracy_score,precision_score


In [3]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *

In [ ]:
!pip install keras-tuner

In [5]:
!pip install protobuf
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [6]:
train_ds='/content/drive/MyDrive/project_neural_net/OUPUT_DATASET/train'
test_ds='/content/drive/MyDrive/project_neural_net/OUPUT_DATASET/test'
val_ds='/content/drive/MyDrive/project_neural_net/OUPUT_DATASET/validation'

train_no_tumor=os.listdir(train_ds+'/no_tumor')
train_glioma_tumor=os.listdir(train_ds+'/glioma_tumor')
train_pituitary_tumor=os.listdir(train_ds+'/pituitary_tumor')
train_meningioma_tumor=os.listdir(train_ds+'/meningioma_tumor')

test_no_tumor=os.listdir(test_ds+'/no_tumor')
test_glioma_tumor=os.listdir(test_ds+'/glioma_tumor')
test_pituitary_tumor=os.listdir(test_ds+'/pituitary_tumor')
test_meningioma_tumor=os.listdir(test_ds+'/meningioma_tumor')

val_no_tumor=os.listdir(val_ds+'/no_tumor')
val_glioma_tumor=os.listdir(val_ds+'/glioma_tumor')
val_pituitary_tumor=os.listdir(val_ds+'/pituitary_tumor')
val_meningioma_tumor=os.listdir(val_ds+'/meningioma_tumor')

**DATA size:**

In [7]:
print('train no tumor size: ',len(train_no_tumor))
print('train glioma tumor size:',len(train_glioma_tumor))
print('train pituitary tumor size: ',len(train_pituitary_tumor))
print('train meningioma tumor size:',len(train_meningioma_tumor))

train no tumor size:  300
train glioma tumor size: 272
train pituitary tumor size:  268
train meningioma tumor size: 272


In [8]:
print('test no tumor size: ',len(test_no_tumor))
print('test glioma tumor size:',len(test_glioma_tumor))
print('test pituitary tumor size: ',len(test_pituitary_tumor))
print('test meningioma tumor size:',len(test_meningioma_tumor))

test no tumor size:  50
test glioma tumor size: 93
test pituitary tumor size:  90
test meningioma tumor size: 94


In [9]:
print('val no tumor size: ',len(val_no_tumor))
print('val glioma tumor size:',len(val_glioma_tumor))
print('val pituitary tumor size: ',len(val_pituitary_tumor))
print('val meningioma tumor size:',len(val_meningioma_tumor))

val no tumor size:  50
val glioma tumor size: 93
val pituitary tumor size:  91
val meningioma tumor size: 94


**Augmentation:**

In [10]:
trainAug = ImageDataGenerator(rescale = 1./255,validation_split = 0.2,                                  
        rotation_range=5,width_shift_range=0.2,height_shift_range=0.2,
        shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True,vertical_flip=True,
        fill_mode='nearest')


valAug=ImageDataGenerator(rescale=1./255, validation_split = 0.2)

testAug=ImageDataGenerator(rescale=1./255 )

**The use of Augmentation:**

In [11]:
train_dataset  = trainAug.flow_from_directory(directory = train_ds,
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   batch_size = 64)

valid_dataset = valAug.flow_from_directory(directory = test_ds,
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  batch_size = 64)

test_dataset = testAug.flow_from_directory(directory = val_ds,
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  batch_size = 64)

Found 1112 images belonging to 4 classes.
Found 327 images belonging to 4 classes.
Found 328 images belonging to 4 classes.


**Print the Tag of datasets:**

In [12]:
import scipy
image_batch_train, label_batch_train = next(iter(train_dataset))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_dataset.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print('0: ',dataset_labels[0])
print('1: ',dataset_labels[1])
print('2: ',dataset_labels[2])
print('3: ',dataset_labels[3])

Image batch shape:  (64, 224, 224, 3)
Label batch shape:  (64, 4)
0:  Glioma_Tumor
1:  Meningioma_Tumor
2:  No_Tumor
3:  Pituitary_Tumor


**HyperParameters Models:**

In [13]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(224,224,3)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.MaxPool2D(
        pool_size = hp.Choice('pool_1_size', values = [3,5])
      ),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(
        filters=hp.Int('conv_3_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
        activation='relu'
    ),

    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(4, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

**Using Random Search to find the optimal model:**

In [18]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=10,directory='/content/drive/MyDrive/project_neural_net/Model/output_new1',project_name="Brain_Tumor")

In [19]:
tuner_search.search(train_dataset ,epochs = 45, callbacks=[EarlyStopping(monitor='val_loss', mode='min',patience=10)], validation_data=(valid_dataset))


Trial 10 Complete [00h 14m 13s]
val_accuracy: 0.6972476840019226

Best val_accuracy So Far: 0.7003058195114136
Total elapsed time: 01h 24m 20s
INFO:tensorflow:Oracle triggered exit


In [20]:
tuner_search.results_summary()

Results summary
Results in /content/drive/MyDrive/project_neural_net/Model/output_new1/Brain_Tumor
Showing 10 best trials
Trial summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 5
conv_2_filter: 64
conv_2_kernel: 5
pool_1_size: 5
conv_3_filter: 64
conv_3_kernel: 5
dense_1_units: 64
learning_rate: 0.001
Score: 0.7003058195114136
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 3
pool_1_size: 3
conv_3_filter: 48
conv_3_kernel: 5
dense_1_units: 32
learning_rate: 0.001
Score: 0.6972476840019226
Trial summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 5
pool_1_size: 5
conv_3_filter: 32
conv_3_kernel: 5
dense_1_units: 96
learning_rate: 0.001
Score: 0.6177369952201843
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 3
pool_1_size: 5
conv_3_filter: 64
conv_3_kernel: 5
dense_1_units: 112
learning_rate: 0.001
Score: 0.60550457239151
Trial summary
H

**Taking the most optimal model:**

In [21]:
model=tuner_search.get_best_models(num_models=1)[0]


**save model:**

In [22]:
model.summary()
model.save('/content/drive/MyDrive/project_neural_net/Model/hyperModel')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 80)      6080      
                                                                 
 conv2d_1 (Conv2D)           (None, 216, 216, 64)      128064    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 43, 43, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 43, 43, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 39, 39, 64)        102464    
                                                                 
 flatten (Flatten)           (None, 97344)             0         
                                                        

**calculation of predict images:**

In [31]:
y_pred=[]
model = keras.models.load_model('/content/drive/MyDrive/project_neural_net/Model/hyperModel')
'''This function loads an image'''

def prepare_img(img, img_width, img_height,channels):
    img =  tf.keras.preprocessing.image.img_to_array(img)
    img = img.reshape(1, img_width, img_height, channels).astype('float32')
    img = img / 255.0
    return img


for img in test_no_tumor:
    img = load_img(test_ds+"/no_tumor/"+ img ,target_size=(224, 224))
    img = prepare_img(img,224,224,3)
    predictions = model.predict(img)
    index_of_higher_value  = predictions[0].argmax()
    y_pred.append(predictions[0].argmax())


for img in test_glioma_tumor:
    img = load_img(test_ds+ "/glioma_tumor/"+img ,target_size=(224, 224))
    img = prepare_img(img,224,224,3)
    predictions = model.predict(img)
    index_of_higher_value  = predictions[0].argmax()
    y_pred.append(predictions[0].argmax())


for img in test_pituitary_tumor:
    img = load_img(test_ds +"/pituitary_tumor/"+ img ,target_size=(224, 224))
    img = prepare_img(img,224,224,3)
    predictions = model.predict(img)
    index_of_higher_value  = predictions[0].argmax()
    y_pred.append(predictions[0].argmax())


for img in test_meningioma_tumor:
    img = load_img(test_ds+ "/meningioma_tumor/"+img,target_size=(224, 224))
    img = prepare_img(img,224,224,3)
    predictions = model.predict(img)
    index_of_higher_value = predictions[0].argmax()
    y_pred.append(predictions[0].argmax())

**calculation of accuracy,recall,precision and f1 score:**

In [33]:
y_test=[2]*len(test_no_tumor)
y_test+=[0]*len(test_glioma_tumor)
y_test+=[3]*len( test_pituitary_tumor)
y_test+=[1]*len(test_meningioma_tumor)

print('Accuracy: ',round(accuracy_score(y_test, y_pred),2))
print('Recall: ',round(recall_score(y_test, y_pred,average='weighted'),2))
print('Precision: ',round(precision_score(y_test, y_pred,average='weighted'),2))
print('f1 score :',round(f1_score(y_test,y_pred, average='weighted'),2))

print(classification_report(y_test, y_pred))

Accuracy:  0.7
Recall:  0.7
Precision:  0.7
f1 score : 0.69
              precision    recall  f1-score   support

           0       0.75      0.61      0.67        93
           1       0.66      0.49      0.56        94
           2       0.63      0.90      0.74        50
           3       0.74      0.90      0.81        90

    accuracy                           0.70       327
   macro avg       0.69      0.73      0.70       327
weighted avg       0.70      0.70      0.69       327



**Visualization of images:**

In [ ]:
import ipyplot
images=[]
for i in os.listdir('./Data_jpg/a'):
  images.append('./Data_jpg/a/'+i)

labels=['adenocarcinoma','adenocarcinoma','adenocarcinoma','adenocarcinoma','adenocarcinoma','adenocarcinoma']
ipyplot.plot_class_tabs(images,labels, img_width=150)



images=[]
for i in os.listdir('./Data_jpg/l'):
  images.append('./Data_jpg/l/'+i)

labels=['large','large','large','large','large','large']
ipyplot.plot_class_tabs(images,labels, img_width=150)

images=[]
for i in os.listdir('./Data_jpg/s'):
  images.append('./Data_jpg/s/'+i)

labels=['squamous','squamous','squamous','squamous','squamous','squamous']
ipyplot.plot_class_tabs(images,labels, img_width=150)


images=[]
for i in os.listdir('./Data_jpg/n'):
  images.append('./Data_jpg/n/'+i)

labels=['normal','normal','normal','normal','normal','normal']
ipyplot.plot_class_tabs(images,labels, img_width=150)